In [2]:
import os
import pandas as pd
import scipy.interpolate
import numpy as np
import matplotlib.pyplot as plt
import warnings
import yaml

class Scan:
      def __init__(self,filename):
            self.theFileName = filename
            self.scanData = []
            self.dataStartIndex = 0
            self.headers = ['Wavelength', self.theFileName.split('.')[0]]
            try:
                  for i, line in enumerate(open(self.theFileName, 'r')):
                        if line == '#DATA\n':
                              self.dataStartIndex = i
                        if self.dataStartIndex != 0:
                              self.scanData.append(line)
                  del self.scanData[0]
                  self.parsedScanData = [l.strip('\n').split('\t') for l in self.scanData]
                  self.df = pd.DataFrame(self.parsedScanData,columns=self.headers).sort_values('Wavelength',ascending=True)
            except:
                  warnings.warn('The file you provided has an error associated with it.')
            
      def data(self):
            #Returns a pandas dataframe containing the file's scan data
            return(self.df)
      
      def filename(self):
            return(self.theFileName)
      
      def quarterWavePoint(self,functionType='max',waveMin=420,waveMax=600):
            wavelength = self.df['Wavelength'].tolist()
            wavelength = [float(i) for i in wavelength]      
            reflection = self.df[self.theFileName.split('.')[0]].tolist() 
            reflection = [float(i) for i in reflection]
            
            t,c,k = scipy.interpolate.splrep(wavelength,reflection)
            spline = scipy.interpolate.BSpline(t,c,k)
            
            numberOfPoints = 100000
            scanWaveMax = max(wavelength)
            scanWaveMin = min(wavelength)
            xx = np.linspace(scanWaveMin,scanWaveMax,numberOfPoints)
            yyFull = spline(xx)
      
            yy = spline(xx[(xx>waveMin)&(xx<waveMax)])
            
            if functionType == 'min':
                  self.theReflection = min(yy)
            if functionType == 'max':
                  self.theReflection = max(yy)
                  
            self.theReflectionIndex = np.where(yy == self.theReflection)[0][0]
            self.theReflectionWavelength = xx[(xx>waveMin)&(xx<waveMax)][self.theReflectionIndex]
            
            #print(self.theReflection)
            #print(self.theReflectionWavelength)
            
            return(self.theReflectionIndex,self.theReflectionWavelength,self.theReflection)
            
      def calculateIndex(self,substrate='BK7',scanLocation=r'C:\Users\Kevin\Documents\GitFiles\visimax\Scans\ReferenceScans'):
            (self.theReflectionIndex, self.theReflectionWavelength,self.theReflection) = self.quarterWavePoint(functionType='max',waveMin=420,waveMax=600)
            
            self.substrateFile = os.path.join(scanLocation,substrate+'.txt')
            self.substrate = pd.read_csv(self.substrateFile,sep='\t')
            
            wavelength = self.substrate['Wavelength'].tolist()
            wavelength = [float(i) for i in wavelength]      
            refractiveIndex = self.substrate['Refractive Index'].tolist() 
            refractiveIndex = [float(i) for i in refractiveIndex]
            
            t,c,k = scipy.interpolate.splrep(wavelength,refractiveIndex)
            spline = scipy.interpolate.BSpline(t,c,k)
            
            numberOfPoints = 100000
            scanWaveMax = max(wavelength)
            scanWaveMin = min(wavelength)
            xx = np.linspace(scanWaveMin,scanWaveMax,numberOfPoints)
            yyFull = spline(xx)
      
            self.substrateIndex = spline(self.theReflectionWavelength)
            
            self.materialRefractiveIndex = ((-self.substrateIndex * ((self.theReflection/100.0)**0.5 + 1))/((self.theReflection/100.0)**0.5 - 1))**0.5
            
            #print('The refractive index of your material is {} at {} nm.'.format(self.materialRefractiveIndex,self.theReflectionWavelength))
            
            return(self.materialRefractiveIndex,self.theReflectionWavelength)
            
      def calculateThickness(self,quarterWaveNumber=3):
            (self.materialRefractiveIndex, self.theReflectionWavelength) = self.calculateIndex()
            self.thickness = (quarterWaveNumber/4)*self.theReflectionWavelength/self.materialRefractiveIndex
            return(self.thickness)
            

folderPath = r'C:\Users\Kevin\Documents\GitFiles\visimax\Scans'
os.chdir(folderPath)
allFiles = os.listdir(folderPath)
fileNames = filter(lambda x: x[-4:] == '.asc', allFiles)
headers = ['Wavelength','']

file = []
thickness = []
index = []
wavelength = []

for filename in fileNames:

      scan = Scan(filename)
      
      file.append(filename)
      thickness.append(scan.calculateThickness())
      index.append(scan.calculateIndex()[0])
      wavelength.append(scan.quarterWavePoint()[1])
      
test = pd.DataFrame(list(zip(file,thickness,index,wavelength)))

In [3]:
test

,0,1,2,3
0,180627-8 bot alpha.Sample.asc,143.195309,2.447868,467.364374
1,180627-8 bot omega.Sample.asc,144.506522,2.442299,470.570906
2,180627-8 mid alpha.Sample.asc,143.780364,2.453971,470.443704
3,180627-8 mid omega.Sample.asc,143.681509,2.456046,470.517905
4,180627-8 top alpha.Sample.asc,138.610441,2.478193,458.004480
5,180627-8 top omega.Sample.asc,138.642437,2.479054,458.269483
6,180627-8 top omega.txt.asc,138.642437,2.479054,458.269483
7,180627-9 bottom alpha.Sample.asc,144.271573,2.436633,468.715887
8,180627-9 bottom omega.Sample.asc,145.154739,2.438513,471.948919
9,180627-9 middle alpha.Sample.asc,143.115729,2.447174,466.972170


Object `scipy.interpolate` not found.
